<a href="https://colab.research.google.com/github/HaoHuangNEU/Finetuned_Model/blob/main/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install \
  langchain_community \
  langchain_pinecone \
  langchain_openai \
  unstructured \
  langchain-text-splitters

In [ ]:
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
import glob

In [ ]:
!pip install unstructured[pdf]

In [ ]:
loader = DirectoryLoader('data',glob="**/*.pdf")

In [ ]:
docs = loader.load()

In [ ]:
docs[0]

Document(metadata={'source': 'data/treatment.pdf'}, page_content='A treatment plan for mental illness is a structured outline created by mental health professionals in collaboration with the patient. It provides a roadmap for managing symptoms, improving mental health, and achieving specific goals. Here’s an overview of the key components of a treatment plan: 1. Assessment and Diagnosis:\n\n\n\n\n\nInitial Evaluation: A thorough assessment is conducted by a mental health professional, which may include interviews, questionnaires, medical history review, and sometimes physical exams. Diagnosis: Based on the assessment, a diagnosis is made, identifying the specific mental health condition(s) affecting the patient.\n\n2. Goals and Objectives:\n\n\n\n\n\nSetting Goals: The patient and therapist work together to identify specific, measurable, and realistic goals for treatment. These can include reducing symptoms, improving daily functioning, and enhancing quality of life. Short-term Objecti

In [ ]:
from google.colab import userdata

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

In [ ]:
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"

)

index_name="pinecone-chatbot"

text_splitter = RecursiveCharacterTextSplitter()
split_docs = text_splitter.split_documents(docs)



In [ ]:
split_docs[1]

Document(metadata={'source': 'data/treatment.pdf'}, page_content='9. Long-term Maintenance:\n\n\n\n\n\nSustaining Progress: The plan should outline how to maintain mental health improvements over time, including ongoing therapy, medication adherence, and regular follow-up appointments. Continued Learning: Encouraging continued self-awareness and learning about mental health to stay informed and prepared for potential challenges.\n\n10. Flexibility and Adaptation:\n\n\n\nAdjusting the Plan: The treatment plan should be flexible, allowing for changes based on the patient’s progress, life circumstances, or new insights gained during therapy.\n\nPatient Involvement:\n\n\n\nEmpowerment: The patient should be actively involved in creating and following the treatment plan. This empowerment fosters a sense of control and responsibility for their mental health.\n\nA treatment plan is a dynamic, collaborative document that evolves with the patient’s needs and progress. It serves as a guide to he

In [ ]:
vectorstore = PineconeVectorStore.from_documents(split_docs,embeddings,index_name = index_name)

In [ ]:
query="how to treat mental illness"

similar_docs = vectorstore.similarity_search(query)

In [ ]:
similar_docs

[Document(metadata={'source': 'data/treatment.pdf'}, page_content='A treatment plan for mental illness is a structured outline created by mental health professionals in collaboration with the patient. It provides a roadmap for managing symptoms, improving mental health, and achieving specific goals. Here’s an overview of the key components of a treatment plan: 1. Assessment and Diagnosis:\n\n\n\n\n\nInitial Evaluation: A thorough assessment is conducted by a mental health professional, which may include interviews, questionnaires, medical history review, and sometimes physical exams. Diagnosis: Based on the assessment, a diagnosis is made, identifying the specific mental health condition(s) affecting the patient.\n\n2. Goals and Objectives:\n\n\n\n\n\nSetting Goals: The patient and therapist work together to identify specific, measurable, and realistic goals for treatment. These can include reducing symptoms, improving daily functioning, and enhancing quality of life. Short-term Object

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

llm = ChatOpenAI(
    model="",
    temperature=0
)

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever()
)

